2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [121]:
import pandas as pd

news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

Наши новости

In [ ]:
print(news.shape)
news.head(3)

Загрузим пользователей и списки последних прочитанных новостей

In [123]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [124]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [125]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [126]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [127]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

151

In [128]:
morph = pymorphy2.MorphAnalyzer()

In [129]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [130]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [131]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

Wall time: 39.1 s


In [132]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 4min 52s


А теперь в 3 строчки обучим нашу модель

In [133]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [134]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [135]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 53.5 s


In [136]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [137]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.096982375), (6, 0.12409564), (12, 0.38193777), (24, 0.3782062)]

In [138]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: организм общество конкурс лаборатория больной который это
topic_1: фильм мужской дальневосточный соль женский информировать семинар
topic_2: обнаружить научный наука расследование ребёнок лечение страдать
topic_3: продукция корея сотрудничать су южный корейский трата
topic_4: млн напомнить пенсия рак мозг составить препарат
topic_5: комплексный родить жуковский джонсон калинин куба расположиться
topic_6: автобус подсчёт мэй употребление таиланд проба грунт
topic_7: рубль дело год суд который уголовный сотрудник
topic_8: банк долг бюджет это карта социальный ресурс
topic_9: следствие палата физика метод девочка регистрация аналог
topic_10: год это который человек время мочь весь
topic_11: год это новый рынок компания проект который
topic_12: украина гражданин украинский киев фронт народный дыра
topic_13: граница орден сочи таможенный ресторан визовый остановка
topic_14: россия год это российский президент который nn
topic_15: депутат убийство который тело человек вицепремьер жу

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [139]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [140]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.062988,0.441555,0.035717,...,0.0,0.0,0.0,0.169426,0.0,0.0,0.0,0.0,0.035041,0.000000
1,4896,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.043679,0.095970,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.300670,0.538646
2,4897,0.096863,0.000000,0.0,0.0,0.000000,0.0,0.124114,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.378318
3,4898,0.000000,0.000000,0.0,0.0,0.025519,0.0,0.075677,0.000000,0.000000,...,0.0,0.0,0.0,0.072397,0.0,0.0,0.0,0.0,0.000000,0.298194
4,4899,0.000000,0.230122,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.144346,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [141]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [142]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [143]:
doc_dict[293622]

array([0.07515377, 0.01680806, 0.        , 0.        , 0.02007674,
       0.        , 0.        , 0.01853702, 0.04989579, 0.        ,
       0.1773452 , 0.07054869, 0.        , 0.03147089, 0.20212735,
       0.        , 0.        , 0.01871527, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.31081808])

In [331]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [332]:
get_user_embedding(user_articles_list)

array([0.01224864, 0.        , 0.        , 0.        , 0.00602282,
       0.        , 0.        , 0.08378848, 0.        , 0.00863222,
       0.16620256, 0.0077173 , 0.01915931, 0.        , 0.29317808,
       0.06525665, 0.01235426, 0.00386927, 0.03521901, 0.        ,
       0.        , 0.        , 0.03427243, 0.03352469, 0.2080056 ])

Интересовался новостями с топиками topic_10, topic_14, topic_16 

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [333]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.082879,0.015263,0.018064,0.0,0.009156,0.0,0.0,0.013217,0.017441,...,0.063738,0.022435,0.011137,0.007425,0.0,0.019625,0.005185,0.075337,0.000000,0.157301
1,u108690,0.019339,0.003064,0.021011,0.0,0.008980,0.0,0.0,0.052087,0.008179,...,0.044971,0.017388,0.021208,0.014162,0.0,0.002427,0.002491,0.064137,0.004679,0.255120
2,u108339,0.010202,0.000000,0.046048,0.0,0.000000,0.0,0.0,0.100681,0.003854,...,0.071514,0.033677,0.006888,0.009163,0.0,0.010635,0.005710,0.091229,0.022170,0.149348


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [334]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [335]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.082879,0.015263,0.018064,0.0,0.009156,0.0,0.0,0.013217,0.017441,...,0.022435,0.011137,0.007425,0.0,0.019625,0.005185,0.075337,0.000000,0.157301,0
1,u108690,0.019339,0.003064,0.021011,0.0,0.008980,0.0,0.0,0.052087,0.008179,...,0.017388,0.021208,0.014162,0.0,0.002427,0.002491,0.064137,0.004679,0.255120,1
2,u108339,0.010202,0.000000,0.046048,0.0,0.000000,0.0,0.0,0.100681,0.003854,...,0.033677,0.006888,0.009163,0.0,0.010635,0.005710,0.091229,0.022170,0.149348,1


In [336]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [337]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [338]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [339]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.16217456, 0.06406532, 0.41084591, 0.36432453, 0.01828016,
       0.07139835, 0.13610266, 0.00839398, 0.087657  , 0.04754025])

In [340]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

### Рассчитаем Precision, Recall, F_score

In [341]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
thresholds_mean = thresholds[ix]
fscore_mean = fscore[ix]
precision_mean = precision[ix]
recall_mean = recall[ix]
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.296278, F-Score=0.687, Precision=0.652, Recall=0.727


In [342]:
roc_auc_score_mean = roc_auc_score(y_test, preds)
roc_auc_score_mean

0.9502924588638874

В целом мы видим, что получившиеся векторные представления содержат какой-то сигнал и позволяют решать нашу прикладную задачу. 

Попробуем заменить параметр на медиану

In [343]:
def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [344]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.0632,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.169917
1,u108690,0.0096,0.0,0.019203,0.0,0.0,0.0,0.0,0.013126,0.0,...,0.018892,0.0,0.0,0.0,0.0,0.0,0.000000,0.055016,0.0000,0.283771
2,u108339,0.0000,0.0,0.045838,0.0,0.0,0.0,0.0,0.074016,0.0,...,0.076991,0.0,0.0,0.0,0.0,0.0,0.005008,0.089500,0.0189,0.160474


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [345]:
target = pd.read_csv("users_churn.csv")
X = pd.merge(user_embeddings, target, 'left')

In [346]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [347]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [348]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.08896189, 0.06661566, 0.63503528, 0.61305158, 0.01245852,
       0.08104417, 0.06599509, 0.00430486, 0.18497644, 0.02484809])

### Рассчитаем Precision, Recall, F_score

In [349]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
roc_auc_score_median = roc_auc_score(y_test, preds)
thresholds_median = thresholds[ix]
fscore_median = fscore[ix]
precision_median = precision[ix]
recall_median = recall[ix]
print(f'Threshold={round(thresholds_median,4)}, F-Score={round(fscore_median, 4)}, Precision={round(precision_median,4)}, Recall={round(recall_median,4)}, roc_auc_score={round(roc_auc_score_median,4)}')

Threshold=0.2917, F-Score=0.7403, Precision=0.6745, Recall=0.8204, roc_auc_score=0.962


С медианным параметром показатели получились лучше

Попробуем заменить параметр на максимальное значение

In [350]:
def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [351]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.274572,0.074769,0.108386,0.0,0.034857,0.0,0.0,0.060762,0.054753,...,0.247581,0.113129,0.048110,0.044549,0.0,0.117751,0.031112,0.244971,0.000000,0.310818
1,u108690,0.050567,0.018385,0.058027,0.0,0.031108,0.0,0.0,0.257723,0.028333,...,0.176728,0.081378,0.103446,0.049562,0.0,0.014562,0.014946,0.158816,0.028072,0.373477
2,u108339,0.031614,0.000000,0.130020,0.0,0.000000,0.0,0.0,0.273916,0.023121,...,0.131823,0.160240,0.041326,0.054978,0.0,0.043785,0.012514,0.219142,0.056947,0.232877


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [352]:
target = pd.read_csv("users_churn.csv")
X = pd.merge(user_embeddings, target, 'left')

In [353]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [354]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [355]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.40990713, 0.00671729, 0.60786523, 0.32602417, 0.02786556,
       0.04992486, 0.07150608, 0.0155952 , 0.0116693 , 0.10223014])

### Рассчитаем Precision, Recall, F_score

In [356]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
roc_auc_score_max = roc_auc_score(y_test, preds)
thresholds_max = thresholds[ix]
fscore_max = fscore[ix]
precision_max = precision[ix]
recall_max = recall[ix]
print(f'Threshold={round(thresholds_max,4)}, F-Score={round(fscore_max, 4)}, Precision={round(precision_max,4)}, Recall={round(recall_max,4)}, roc_auc_score={round(roc_auc_score_max,4)}')

Threshold=0.3426, F-Score=0.748, Precision=0.7449, Recall=0.751, roc_auc_score=0.9619


In [357]:
result_table = pd.DataFrame({'fscore':[fscore_mean, fscore_median, fscore_max], 'Precision':[precision_mean, precision_median, precision_max], 'Recall':[recall_mean, recall_median, recall_max], 'roc_auc_score':[roc_auc_score_mean, roc_auc_score_median, roc_auc_score_max]}, index=['mean', 'median', 'max'])
result_table 

,fscore,Precision,Recall,roc_auc_score
mean,0.687259,0.652015,0.726531,0.950292
median,0.740331,0.674497,0.820408,0.961958
max,0.747967,0.744939,0.751020,0.961893


С каждым дальнейшим методом у нас растет точность, но снижается полнота. Использование median и max оказались более удачными с точки зрения fscore и roc_auc_score. Median и max скорее всего больше отражают реальность. При использовании median коэффициенты заинтересованности пользователя меньше, есть 0, которые попадают в медианные значения. Для модели существеннее разница между интересами пользователя, точность растет, но и полноста падает, т.к. меньше пользователей попадает в выборку target. В общем метрики лучше. С использованием max - общая заинтересованность пользователя во всех темах выше, так как берутся максимальные значения. Больше данных, но и отличия между интересами к определенным темам не так кардинальны. Судя по всему, Median в данном случае подходит больше.